In [11]:
import math
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as mpatches
import os
import glob
import numpy as np
from typing import Iterable
import pickle
from tqdm.notebook import tqdm
import tensorflow as tf
from scipy.fft import fft, ifft
from statsmodels.tsa.stattools import acf, pacf
from sklearn.decomposition import PCA

import keras
import keras_tuner as kt
from kerastuner import RandomSearch
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras import regularizers

In [12]:
# load a single shot using pandas parquet file reader 
def load_shot(shotnum: int) -> pd.DataFrame:
    shot = pd.read_parquet("../data/DISR_data/JET_DATAno" + str(shotnum) + ".parquet")
    return shot

df_shot = load_shot(96885)
df_shot['IPE'] = df_shot['IPLA'] - df_shot['IPref']
display(df_shot)

,time,IPLA,IPref,ECEcore,SSXcore,LI,Q95,ZMAG,Vloop,IPE
0,0.022000,-53160.492188,0.022461,1141.500610,0.002289,0.661752,10.408838,0.020125,-17.961126,-53160.515625
1,0.023000,-56066.082031,0.022461,1141.500610,0.001373,0.661752,10.408838,0.020125,-17.642231,-56066.105469
2,0.024000,-59827.996094,0.022461,1141.500610,0.000916,0.661752,10.408838,0.020125,-18.239368,-59828.019531
3,0.025000,-63139.011719,0.022461,1141.500610,0.001679,0.661752,10.408838,0.020125,-18.200048,-63139.035156
4,0.026000,-66272.710938,0.022461,1141.500610,0.001221,0.661752,10.408838,0.020125,-17.801512,-66272.734375
...,...,...,...,...,...,...,...,...,...,...
16880,16.902000,-100682.593750,0.022461,1224.449097,0.002441,1.155780,3.558453,0.329814,-55.337883,-100682.617188
16881,16.903000,-96515.125000,0.022461,1224.449097,0.001831,1.155780,3.558453,0.329814,-49.116920,-96515.148438
16882,16.903999,-85514.335938,0.022461,1224.449097,0.002441,1.155780,3.558453,0.329814,-43.695698,-85514.359375
16883,16.905001,-70803.953125,0.022461,1224.449097,0.002899,1.155780,3.558453,0.329814,-37.757305,-70803.976562


In [13]:
model = tf.keras.models.load_model("../models/nn_window.keras")

window = 0

# Add 'Instability' and 'Window' attributes to the DataFrame
df_shot['Instability'] = 0
df_shot['Window'] = window
df_shot['Label'] = -1

# Define the window size and the number of samples with the same label required
window_size = 20
window_cursor_one = 0
window_cursor_two = window_size - 1
label_count = 15

# Iterate over the samples in the DataFrame
for i in tqdm(range(len(df_shot)), desc='Processing samples'):
    # Get the current sample
    sample = df_shot.iloc[i]
    
    # Perform the prediction using the Keras model
    prediction = model.predict(np.array([sample]))[0]
    label = np.argmax(prediction)

    df_shot.loc[i, 'Label'] = label

    max_label = df_shot['Label'].iloc[window_cursor_one : window_cursor_two].value_counts().max()
    
    # Check if there are enough samples with the same label in the window
    if max_label >= label_count and max_label != -1:
        window += 1
        # Set the window attribute for the samples inside the window
        df_shot.loc[window_cursor_one : window_cursor_two, 'Window'] = window
        
        # Move the window to the next 20 samples
        window_cursor_one += window_size
        window_cursor_two += window_size

        # Check if the label is 1
        if label == 1:
            # Increase the instability by 1 for all following samples
            df_shot.loc[i + 1:, 'Instability'] += 1

    elif i == window_cursor_two:
        # Move the window to the next samples
        window_cursor_one += 1
        window_cursor_two += 1

# Display the updated DataFrame
display(df_shot)

Processing samples:   0%|          | 0/16885 [00:00<?, ?it/s]

1/1 [==============================] - 0s 19ms/step


In [ ]:
# Retrieve the unique window values different than 0 and where the label is equal to 1
window = df_shot[(df_shot['Window'] != 0) & (df_shot['Label'] == 1)]['Window'].unique()

# Print the dataframe window per window
display(df_shot[df_shot['Window'] == window[0]])